# Import libraries

In [ ]:
import pandas as pd
pd.options.display.max_columns = 200

import requests as req
import json
import boto3
import io

import sys
#import logging
import os
#logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)
import random

from functools import reduce
from collections import defaultdict

# Constants

In [ ]:
API_KEY = os.environ.get('rw_api_key', None)
CARTO_USER = 'wri-rw'#os.environ.get('CARTO_USER')
CARTO_KEY = ''#os.environ.get('CARTO_KEY')

# Load data, layer config information

In [ ]:
# Read in data sets info from config file
layer_config = pd.read_csv('/Users/nathansuberi/Desktop/RW_Data/layer_making_tasks/layer_these.csv')
layer_config = layer_config.set_index('wri_id')
layer_config

In [ ]:
cc = cartoframes.CartoContext(base_url='https://{}.carto.com/'.format(CARTO_USER),
                              api_key=CARTO_KEY)

# Load data sets into memory for processing
def load_data(obj, elem):
    print(elem)
    wri_id = elem[0].strip()
    rw_id = elem[1].strip()
    try:
        table_name = carto_data.loc[rw_id]['table_name']
        obj[wri_id] = cc.read(table_name)
        print('Table shape: {}'.format(obj[wri_id].shape))
    except:
        obj[wri_id] = 'Unavailable'
        print('Unavailable')
    return obj

data_tables = reduce(load_data, zip(georef_config.index,georef_config['rw_id']), {})

# Examining existing layers

In [ ]:
def grab_layer_defs(obj, elem):
    print(elem)
    wri_id = elem[0].strip()
    rw_id = elem[1].strip()
    try:
        obj[wri_id] = carto_data.loc[rw_id]['layers']
        print('Num layers: {}'.format(len(obj[wri_id])))
    except:
        obj[wri_id] = 'No Layers'
        print('Layers Unavailable')
    return obj

data_layers = reduce(grab_layer_defs, zip(georef_config.index,georef_config['rw_id']), {})
data_layers

# Find which layers already have the join language in them

def find_join_sql(obj, ds):
    rw_id = ds[0]
    layers = ds[1]
    try:
        obj[rw_id] = [layerdef['attributes']['layerConfig']['body']['layers'][0]['options']['sql'] for layerdef in layers]
    except:
        obj[rw_id] = None 
    
    return obj

def find_interaction(obj, ds):
    rw_id = ds[0]
    layers = ds[1]
    try:
        obj[rw_id] = [layerdef['attributes']['interactionConfig']['output'] for layerdef in layers]
    except:
        obj[rw_id] = None
    
    return obj

sql_to_change = reduce(find_join_sql, data_layers.items(), {})
print("Existing SQL:")
print(sql_to_change)

print()
interaction_configs = reduce(find_interaction, data_layers.items(), {})
print("Existing interaction config:")
print(interaction_configs)

# Generate new layers

In [ ]:
# Augment needed layers w/ correct SQL and Interaction config

## REDUCE
def setup_interaction_config(obj, col, ds):
    _type = str(ds[col].dtype)
    if _type == 'object':
        _type = 'string'
    if _type in ['int64', 'float64']:
        _type = 'numeric'
        
    template = {'column': col,
      'format': None,
      'prefix': '',
      'property': col,
      'suffix': '',
      'type': _type}
    
    obj.append(template)
    return obj

def gen_sql(data_col, table_name, date_col, year, filter_col, filter_val):
    basesql = ('SELECT wri.cartodb_id, ST_Transform(wri.the_geom, 3857)' +
    ' AS the_geom_webmercator, rw_country_name,' +
    ' EXTRACT(YEAR FROM data.{}) AS year,'+
    ' data.{} FROM {} data'+
    ' LEFT OUTER JOIN wri_countries_a wri' +
    ' ON data.rw_country_code = wri.iso_a3' + 
    ' WHERE {} IS NOT NULL AND EXTRACT(YEAR FROM {})={}' )
    if filter_col:
        sql = basesql + ' AND {} ILIKE {}'
        return sql.format(date_col, data_col, table_name, data_col, date_col, year, filter_col, filter_val)
    else:
        return basesql.format(date_col, data_col, table_name, data_col, date_col, year)


def pick_color(ix):
    poss = '0123456789abcdef'
    col = '#'
    for i in range(6):
        col += random.choice(poss)
    return col
    
def gen_cartocss_legend(col, ds):
    _type = str(ds[col].dtype)
    if _type == 'object':
        _type = 'string'
    if _type in ['int64', 'float64']:
        _type = 'numeric'
        
    print('Type of data: {}'.format(_type))
    if _type == 'string':
        breaks = list(ds[col].unique())
        colors = list(map(pick_color, range(len(breaks))))
        
        cartocss = '#table {polygon-opacity: 1; line-width: 0.5; line-color: #FFF; line-opacity: 1;}'   
        legend = []
        for brk, clr in zip(breaks, colors):
            cartocss += ' [{} = {}]?1polygon-fill:{} ;?2'.format(col,brk,clr)
            legend.append({'color':clr, 'name':brk})
        cartocss = cartocss.replace('?1', '{').replace('?2', '}')
        
    if _type == 'numeric':
        # Make breaks by quintiles
        q0 = ds[col].quantile(0)
        q20 = ds[col].quantile(.20)
        q40 = ds[col].quantile(.40)
        q60 = ds[col].quantile(.60)
        q80 = ds[col].quantile(.80)
        q100 = ds[col].quantile(1.00)
        breaks = [q0,q20,q40,q60,q80,q100]
        colors = list(map(pick_color, range(len(breaks))))
        cartocss = '#table {polygon-opacity: 1; line-width: 0.5; line-color: #FFF; line-opacity: 1;}'
        cartocss += ' [{} > {}]?1polygon-fill:{} ;?2'.format(col,breaks[0],colors[0])
        legend = [{'color':colors[0], 'name':'>{}'.format(breaks[0])}]
        for i in range(1,len(breaks)-1):
            cartocss += ' [{} > {}][{} < {}]?1polygon-fill:{} ;?2'.format(col,breaks[i],col,breaks[i+1],colors[i])
            legend.append({'color':colors[i], 'name':'<{}'.format(breaks[i+1])})
        cartocss = cartocss.replace('?1', '{').replace('?2', '}')
       
    return cartocss, legend


### Generating entire layer def

def gen_layer(wri_id, data_col, date_col, year, slider, filter_col, filter_val):
    cartocss, legend = gen_cartocss_legend(data_col, data_tables[wri_id])
    rw_id = georef_config.loc[wri_id, 'rw_id'].strip()
    table_name = carto_data.loc[rw_id]['table_name']
    layer_template = {'attributes': {'application': ['rw'],
      'applicationConfig': {},
      'dataset': georef_config.loc[wri_id,'rw_id'],
      'default': False,
      'description': '',
      'env': 'production',
      'interactionConfig': reduce(lambda obj, col: setup_interaction_config(obj, col, data_tables[wri_id]), data_tables[wri_id].columns, []),
      'iso': [],
      'layerConfig': {'account': 'wri-rw',
       'body': {'layers': [{'options': {'cartocss': cartocss,
           'cartocss_version': '2.3.0',
           'sql': gen_sql(data_col, table_name, date_col, year, filter_col, filter_val)},
          'type': 'mapnik'}],
        'maxzoom': 18,
        'minzoom': 3}},
      'legendConfig': {'items': legend,
       'type': 'choropleth'},
      'name': '',
      'protected': False,
      'provider': 'cartodb'},
     'id': '',
     'type': 'layer'}

    if slider:
        layer_template['attributes']['layerConfig']['timeline'] = True
        layer_template['attributes']['layerConfig']['order'] = int(year)
        layer_template['attributes']['layerConfig']['timelineLabel'] = str(year)
   
    return layer_template
        
layers = defaultdict(list)

layer_prep = {
    'com.009':
}

for wri_id, data in georefed_data.items():
    if type(data) == str:
        continue 
        
    print(wri_id)
    print(data.columns)
    data_col = input('data column?')
    if not data_col:
        layers[wri_id] = 'Skipping'
        continue
    
    date_col = input('date column?')
    slider = input('make slider? type anything for yes')
    slider = True if slider else False
    filter_col = input('filter col?')
    filter_col = filter_col if filter_col else None
    if filter_col:
        filter_opts = [fltr for fltr in data[filter_col].unique() if fltr]
    else:
        filter_opts = [None]
        
    print('Available dates: {}'.format(sorted(data[date_col].unique())))
   
    flag = True
    while flag:
        year = input('year for layer?')
        for fltr in filter_opts:
            layer = gen_layer(wri_id, data_col, date_col, year, slider, filter_col, fltr)
            layers[wri_id].append(layer)

        if slider:
            flag = input('Make another layer for this data set? type anything for yes')
            flag = True if flag else False
        else:
            flag = False

# Examining a particular layer

In [ ]:
carto_data.loc[carto_data['name'].str.lower().str.contains('conflict')]
# aliases set in Interaction Config
carto_data.loc['ea208a8b-4559-434b-82ee-95e041596a3a', 'layers']
# aliases set in dataset
carto_data.loc['ea208a8b-4559-434b-82ee-95e041596a3a']

url = "https://api.resourcewatch.org/v1/dataset/ea208a8b-4559-434b-82ee-95e041596a3a?sort=slug,-provider,userId&status=saved&includes=metadata,vocabulary,widget,layer"

# page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API
payload = { "application":"rw", "page[size]": 1000}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
data = res.json()["data"]
data